# Linear Unmixing of Spectral Fluorescence Microscopy Data

In this notebook we perform linear unmixing on spectral data from Fluorescence Microscopy.

In this case, for a given pixel, we suppose to have a set of intensity measurements at different wavelengths, e.g., $y = [y(\lambda_1),y(\lambda_2),\dots,y(\lambda_n)]$, with $n=32$ for instance. For each one of these spectral bands $\lambda_i$, with $i=1,\dots,n$, and for each fluorophore $f$, with $f=1,\dots,m$, we assume the reference spectra $R_f=[R_f(\lambda_1), R_f(\lambda_2), \dots, R_f(\lambda_n)]$ to be known. 

### 1. Data Preparation

Load mixed image & metadata:

In [ ]:
import os
import json
import tifffile as tiff
import numpy as np

DATA_DIR = '/group/jug/federico/microsim/sim_spectral_data/240717_v0'
load_mip = False

In [ ]:
mixed_opt_img = tiff.imread(
    os.path.join(
        DATA_DIR, 
        f"{"mips" if load_mip else "imgs"}/optical_mixed{"_mip" if load_mip else ""}.tif"
    )
)
print("Loaded optical mixed image!")

In [ ]:
mixed_digital_img = tiff.imread(
    os.path.join(
        DATA_DIR, 
        f"{"mips" if load_mip else "imgs"}/digital_mixed{"_mip" if load_mip else ""}.tif"
    )
)
print("Loaded digital mixed image!")

In [ ]:
with open(os.path.join(DATA_DIR, "sim_coords.json"), "r") as f:
    coords_metadata = json.load(f)

try:    
    with open(os.path.join(DATA_DIR, "sim_metadata.json"), "r") as f:
        sim_metadata = json.load(f)
except FileNotFoundError as e:
    print("Metadata file not found!")
    sim_metadata = None

In [ ]:
# Load GT
gt_img = tiff.imread(os.path.join(DATA_DIR, "ground_truth_img.tif"))

In [ ]:
from utils import coarsen_img

try:
    downscaling = int(sim_metadata["downscale"])
except:
    downscaling = 2
gt_img_downsc = coarsen_img(gt_img, downscaling)

In [ ]:
mixed_opt_img.shape, mixed_digital_img.shape, gt_img.shape, gt_img_downsc.shape, coords_metadata.keys(), sim_metadata.keys()

In [ ]:
for k, v in sim_metadata.items():
    print(f"{k}: {v}")

Compute *PSNR* for the Digital Image w.r.t. the downscaled optical image

In [ ]:
try:
    downscaling = int(sim_metadata["downscale"])
except:
    downscaling = 2
mixed_opt_img_downsc = coarsen_img(mixed_opt_img, downscaling)

In [ ]:
from utils.metrics import SpectralPSNR

dig_psnr = SpectralPSNR(gt=mixed_opt_img_downsc, pred=mixed_digital_img, range_inv=True)
print(f"PSNR digital wrt optical: {dig_psnr:.2f}")

Get reference spectra from `FPBase` using `microsim` API:

In [ ]:
from data.FPData import FPRefMatrix

fp_ref_matrix = FPRefMatrix(sim_metadata["fluorophores"], coords_metadata["w_bins"])
fp_ref_matrix = fp_ref_matrix.create()

**OBSERVATION**
The mixed image is a 16bit image (range: 0-6.5e4), whereas the intensity of fluorophores emission spectra ranges in 0-1 before the binning.

Intuitively, intensity ranges should be the same. However does this really matter?

In my understanding, the answer is NO. Let's see why:

- Suppose the case of 0-1 range normalization. In that case, normalization is obtained by simply dividing each pixel's intensity by the maximum intensity in the image. In other terms we basically divide by a scalar. Therefore, supposing that we normalize in this way both the mixed image and the reference spectra, the linear system becomes:

\begin{equation}
\frac{1}{k_I}y = \frac{1}{k_R}\mathbf{R}c
\end{equation}

where $k_I$ and $k_R$ are scalar. Therefore the solution of this system is the same up to some multiplicative constants.

Therefore, we can normalize everything in the range 0-1 so that quantities are in the same scale.

### 2. Compute the LS solution

In [ ]:
# Normalize image to unmix
mixed_digital_img = (mixed_digital_img - mixed_digital_img.min()) / (mixed_digital_img.max() - mixed_digital_img.min())

In [ ]:
from methods import LeastSquares

# Solving LS for digital image
ls = LeastSquares(mixed_digital_img, fp_ref_matrix)
fp_conc_img_LS = ls.solve()

In [ ]:
from methods import FCLSU

fclsu = FCLSU(mixed_digital_img, fp_ref_matrix)
fp_conc_img_FCLSU = fclsu.solve()

### 3. Visualizing results

In [ ]:
# Load GT (unmixed optical image)
fluor1_gt_img = tiff.imread(os.path.join(DATA_DIR, "imgs/optical_fluor1_gt.tif"))
fluor2_gt_img = tiff.imread(os.path.join(DATA_DIR, "imgs/optical_fluor2_gt.tif"))
fluor3_gt_img = tiff.imread(os.path.join(DATA_DIR, "imgs/optical_fluor3_gt.tif"))

# Add channel dimension
fluor1_gt_img = fluor1_gt_img[np.newaxis, ...]
fluor2_gt_img = fluor2_gt_img[np.newaxis, ...]
fluor3_gt_img = fluor3_gt_img[np.newaxis, ...]

In [ ]:
# Get downsclaed GT
try:
    downscaling = int(sim_metadata["downscale"])
except:
    downscaling = 2
fluor1_gt_img_downsc = coarsen_img(fluor1_gt_img, downscaling)
fluor2_gt_img_downsc = coarsen_img(fluor2_gt_img, downscaling)
fluor3_gt_img_downsc = coarsen_img(fluor3_gt_img, downscaling)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("MIP of Optical (clean) image - Ground Truth", fontsize=16)
ax[0].imshow(fluor1_gt_img_downsc.max(axis=1)[0, ...])
ax[1].imshow(fluor2_gt_img_downsc.max(axis=1)[0, ...])
ax[2].imshow(fluor3_gt_img_downsc.max(axis=1)[0, ...])


fig, ax = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("MIP of LS solutions from Digital (noisy) image", fontsize=16)
if load_mip:
    ax[0].imshow(fp_conc_img_LS[0, :, :])
    ax[1].imshow(fp_conc_img_LS[1, :, :])
    ax[2].imshow(fp_conc_img_LS[2, :, :])
else:
    ax[0].imshow(fp_conc_img_LS.max(axis=1)[0, :, :])
    ax[1].imshow(fp_conc_img_LS.max(axis=1)[1, :, :])
    ax[2].imshow(fp_conc_img_LS.max(axis=1)[2, :, :])
    
# fig, ax = plt.subplots(1, 3, figsize=(15, 5))
# fig.suptitle("MIP of FCLSU solutions from Digital (noisy) image", fontsize=16)
# if load_mip:
#     ax[0].imshow(fp_conc_img_FCLSU[0, :, :])
#     ax[1].imshow(fp_conc_img_FCLSU[1, :, :])
#     ax[2].imshow(fp_conc_img_FCLSU[2, :, :])
# else:
#     ax[0].imshow(fp_conc_img_FCLSU.max(axis=1)[0, :, :])
#     ax[1].imshow(fp_conc_img_FCLSU.max(axis=1)[1, :, :])
#     ax[2].imshow(fp_conc_img_FCLSU.max(axis=1)[2, :, :])

### 4. Evaluation

Compute error with respect to ground truth images for each flurophore.

The ground truth images are `(Z, Y, X)` arrays, which correspond to the clean image that we would get from one FP alone.

Observe that the solution of the linear unmixing problem expresses **concentrations** of FPs, whereas the GT microscope images express values as intensities. <br>
A way to compare these quantities is by using the **Range Invariant PSNR**, which is independent of the scale of the involved quantities.   

Here, we compute the overall error for each FP as the Range Invariant PSNR and visualize it:

In [ ]:
from utils.plots import plot_unmixed_vs_gt

plot_unmixed_vs_gt(
    gt_img=[fluor1_gt_img_downsc, fluor2_gt_img_downsc, fluor3_gt_img_downsc],
    unmixed_img=fp_conc_img,
    method="LS",    
)